In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
from sklearnex import patch_sklearn
patch_sklearn()

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from pathlib import Path
import h5py
import pickle
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from skimage.transform import resize
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from src.data.file_utils import GetTV
from src.models import model, predict

from IPython.display import clear_output

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [4]:
models = predict.list_models()
for idx, model in enumerate(models):
    print(idx, model)

0 ../models/lr_tv_inv_outer.pkl
1 ../models/weighted_outer_all.pkl
2 ../models/lr_inversion_manual.pkl
3 ../models/lr.pkl
4 ../models/lr_supervised_mdl.pkl


# Data Prep

In [5]:
run_type = 'all'
prep_filename = 'weighted_outer_dataset_' + run_type
prediction_filename = 'weighted_outer_' + run_type
algorithm = 'linear'
split_ratio = 0.2

In [6]:
tv_path = Path('../data/raw/tv_images/all')
label_path = Path('../data/labels/weighted_emission/all')
prep_path = Path('../data/processed/hdf5')
model_path = Path('../models')

In [7]:
tv = GetTV(tv_path)
files = tv.list_files()
file_lengths = tv.file_lengths()
cumulative_lengths = np.insert(np.cumsum(file_lengths), 0, 0)
tv_dim = tv.load(files[0], 'vid').shape

### Fill Out TV Image Data

In [18]:
def file_split_compile_prep(split_ratio,
                      prep_path,
                      prep_filename,
                      label_path,
                      files):
    # split by file
    train_files, test_files = train_test_split(files, test_size=split_ratio, random_state=182)
    [print(file.stem) for file in train_files]
    print('-------------------')
    [print(file.stem) for file in test_files]
    train_files = files

    points_train = []
    tv_train = []
    points_test = []
    tv_test = []

    for _, file in enumerate(train_files):
        frames = tv.load(file, 'frames').astype('int')
        tv_image = tv.load(file, 'vid')[frames]
        
        test_label_file = (label_path / file.stem).with_suffix('.pkl')
        with open(test_label_file, 'rb') as f:
            labels = pickle.load(f)
            
        for i in range(len(frames)):
            tv_train.append(tv_image[i])
            points_train.append(labels[i])
            
    for _, file in enumerate(test_files):
        frames = tv.load(file, 'frames').astype('int')
        tv_image = tv.load(file, 'vid')[frames]
        
        test_label_file = (label_path / file.stem).with_suffix('.pkl')
        with open(test_label_file, 'rb') as f:
            labels = pickle.load(f)
            
        for i in range(len(frames)):
            tv_test.append(tv_image[i])
            points_test.append(labels[i])
            
    with h5py.File(prep_path / str(prep_filename + '.h5'), 'w') as f:
        f.create_dataset('vid_train', data=tv_train)
        f.create_dataset('points_train', data=points_train)
        f.create_dataset('vid_test', data=tv_test)
        f.create_dataset('points_test', data=points_test)
    print(str(prep_path / str(prep_filename + '.h5')),'has been saved!')

In [19]:
file_split_compile_prep(split_ratio, prep_path, prep_filename, label_path, files)

emission_structure_pu_cam240perp_189448
emission_structure_pu_cam240perp_199171
emission_structure_pu_cam240perp_190113
emission_structure_pu_cam240perp_189100
emission_structure_pu_cam240perp_189451
emission_structure_pu_cam240perp_189057
emission_structure_pu_cam240perp_189093
emission_structure_pu_cam240perp_190115
emission_structure_pu_cam240perp_189081
emission_structure_pu_cam240perp_189101
emission_structure_pu_cam240perp_199166
emission_structure_pu_cam240perp_189061
emission_structure_pu_cam240perp_189094
emission_structure_pu_cam240perp_199172
emission_structure_pu_cam240perp_189090
emission_structure_pu_cam240perp_199351
emission_structure_pu_cam240perp_189062
emission_structure_pu_cam240perp_190116
emission_structure_pu_cam240perp_189097
emission_structure_pu_cam240perp_199354
-------------------
emission_structure_pu_cam240perp_199353
emission_structure_pu_cam240perp_199352
emission_structure_pu_cam240perp_190114
emission_structure_pu_cam240perp_190110
emission_structure_p

# Train Test Deploy

### Train

In [20]:
flatten = lambda x : x.reshape(len(x), -1)

In [21]:
file_name = prep_filename + '.h5'

with h5py.File(prep_path / file_name, 'r') as f:
    vid_train = f['vid_train'][:]
    points_train = f['points_train'][:]
    vid_test = f['vid_test'][:]
    points_test = f['points_test'][:]

files = tv.list_files()
elevation = tv.load(files[0], 'elevation')[0]
radii = tv.load(files[0], 'radii')[0]
vid_shape = tv.load(files[0], 'vid')[0].shape

In [22]:
X_train = flatten(vid_train)
X_test = flatten(vid_test)
y_train = points_train # flatten(points_train)
y_test = points_test # flatten(points_test)

In [23]:
mdl = LinearRegression()
mdl.fit(X_train, y_train)

LinearRegression()

### Test

In [100]:
r_predict = mdl.predict(X_test)
# real_predict = np.array([radii[np.round(r_predict[:,0]).astype(int)],elevation[np.round(r_predict[:,1]).astype(int)]]).T * 100 # from meter to cm (dimension is n x 2)
# real_y = np.array([radii[np.round(y_test[:,0]).astype(int)],elevation[np.round(y_test[:,1]).astype(int)]]).T * 100
err = mean_squared_error(r_predict,y_test) * 100
print(err)
# z_err = np.abs(real_predict[:,1] - real_y[:,1])
# print(f"RMS (cm) : {np.sqrt(np.square(dist).mean())}")
# print(f"Z RMS (cm) : {np.sqrt(np.square(z_err).mean())}")

1.7118457753181309


### Deploy

In [24]:
pickle.dump(mdl, open(f"{model_path / prediction_filename}.pkl", 'wb'))

In [24]:
weight_ml_point_save_path = Path('../data/processed/weight_ml_point')
weight_ml_point_save_path.mkdir(parents=True, exist_ok=True)

In [27]:
for file in files:
    
    clear_output(wait=True)
    
    print('Shot:', file.stem.split('_')[-1])
    point_save_name = weight_ml_point_save_path / f"{file.stem}.pkl"
    label_file = (label_path / file.stem).with_suffix('.pkl')
    with open(label_file, 'rb') as f:
        labels_cartesian = pickle.load(f)
    frames = tv.load(file, 'frames').astype('int')
    tv_image = tv.load(file, 'vid')[frames]
    tv_flatten = flatten(tv_image)
    
    prediction_cartesian = mdl.predict(tv_flatten)
    pickle.dump(prediction_cartesian, open(point_save_name, 'wb'))

Shot: 199354


# Save Video

In [12]:
def nearest_index(array, value):
    """Find the index of the nearest value in an array."""
    return (np.abs(array - value)).argmin()

def get_index(arr, coord):
    # Get the insertion indices
    ind = np.searchsorted(coord, arr)

    # Correct the indices to point to the nearest actual index
    ind = np.clip(ind, 0, len(coord) - 1)

    # Now, adjust the indices to get the closest value
    for i, cval in enumerate(arr):
        if ind[i] > 0 and abs(cval - coord[ind[i] - 1]) < abs(cval - coord[ind[i]]):
            ind[i] -= 1
            
    return ind

In [13]:
flatten = lambda x : x.reshape(len(x), -1)

In [18]:
tv = GetTV(tv_path)
files = tv.list_files()
file_lengths = tv.file_lengths()
cumulative_lengths = np.insert(np.cumsum(file_lengths), 0, 0)
tv_dim = tv.load(files[0], 'vid').shape

with open(f"{model_path / prediction_filename}.pkl", 'rb') as f:
    mdl = pickle.load(f)
    
mp4_save_path = Path('../outputs/video/weighted_ml') / run_type
mp4_save_path.mkdir(parents=True, exist_ok=True)

In [23]:
for file in files:
    
    clear_output(wait=True)
    
    print('Shot:', file.stem.split('_')[-1])
    mp4_save_name = mp4_save_path / f"{file.stem.split('_')[-1]}.mp4"
    label_file = (label_path / file.stem).with_suffix('.pkl')
    with open(label_file, 'rb') as f:
        labels_cartesian = pickle.load(f)
    frames = tv.load(file, 'frames').astype('int')
    tv_image = tv.load(file, 'vid')[frames]
    tv_flatten = flatten(tv_image)
    inverted = tv.load(file, 'inverted')
    elevation = tv.load(file, 'elevation')[0]
    
    prediction_cartesian = mdl.predict(tv_flatten)
    
    prediction = get_index(prediction_cartesian, elevation, )
    labels = get_index(labels_cartesian, elevation)

    # Initialize figure and axes
    print("Animating...")
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    
    img = ax.imshow(inverted[0], origin='lower')
    hline_label = ax.axhline(labels[0], c='lime', label='label')
    hline_prediction = ax.axhline(prediction[0], c='red', label='prediction', ls='--')
    ax.legend(loc='upper right')
    ax.set_title(f'Inverted View: 0')

    fig.suptitle(f"Shot {file.stem.split('_')[-1]}")
    frames = []
    # Function to update the plot
    def update(idx):
        img.set_data(inverted[idx])
        
        hline_label.set_ydata([labels[idx]])
        hline_prediction.set_ydata([prediction[idx]])
        
        ax.set_title(f'Inverted View: {idx}')
        
        return img, hline_label, hline_prediction
        
    # Create the animation using FuncAnimation
    ani = animation.FuncAnimation(fig, update, frames=range(inverted.shape[0]), blit=True, repeat=False)

    # Save the animation as an MP4 file
    print("Saving MP4...")
    FFwriter = animation.FFMpegWriter(fps=30, extra_args=["-vcodec", "libx264"])
    ani.save(mp4_save_name, writer=FFwriter)

    plt.close(fig)

Shot: 199354
Animating...
Saving MP4...
